## Paso 1: Instalar las dependencias y configurar el entorno

In [1]:
!pip install pandas-datareader
!pip install yfinance --upgrade --no-cache-dir

## Paso 2: Importar las dependencias del proyecto

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import datetime as dt
import yfinance as yf 
import contextlib

from tqdm import tqdm_notebook, tqdm #Visualizar porcentaje de completacion de la tarea
from collections import deque #Experience replay

In [3]:
tf.__version__

'2.12.0'

## Paso 3: Construir la red neuronal de la AI del Trader 

In [4]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"):
    
    self.state_size = state_size      #Tamaño de los estados de la AI
    self.action_space = action_space  #Numero de acciones que a a poder tomar [Mantener, comprar, vender]
    self.memory = deque(maxlen=2000)  #Tamaño de la memoria 
    self.inventory = []               #Inentario de divisas que compramos y vendemos
    self.model_name = model_name      #Nombre del modelo
    
    self.gamma = 0.95                 #Gamma de la ecucacion de Bellman
    self.epsilon = 1.0                #Valor de inicio (seleccion de accion)
    self.epsilon_final = 0.01         #Valor del final (seleccion de accion)
    self.epsilon_decay = 0.995        #Decaer continuamente
    
    self.model = self.model_builder()
    
  def model_builder(self):  #Modelo
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))#Numero continuo
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))#Error cuadrado medio (valor que perdemos)
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon: #Decision aleatoria
      return random.randrange(self.action_space) # 0,1 o 2
    
    actions = self.model.predict(state,verbose=0)#Prediccion con base al estado actual
    return np.argmax(actions[0])#Escogemos mayor

  def batch_train(self, batch_size): #Tamaño del bloque de entrenamiento
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)): #Obtenemos los ultimos datos de la memoria
      batch.append(self.memory[i])
    
    for state, action, reward, next_state, done in batch:#Estado donde estaba el agente, accion, recompensa obtenida, siguiente estado, finalizo o no
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state,verbose=0)[0])#Valor maximo
        
      target = self.model.predict(state,verbose=0)
      target[0][action] = reward#Actualizacion de recompensa
      # Temporarily suppress printing during fitting
      with contextlib.redirect_stdout(None):
        self.model.fit(state, target, epochs=1, verbose=0)#Backpropagation
      
    if self.epsilon > self.epsilon_final:#Decaemos epsilon
      self.epsilon *= self.epsilon_decay

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [5]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [6]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [8]:
def dataset_loader(stock_name):
    # Complete the dataset loader function
    start = dt.datetime(2021,1,1)
    end =  dt.date.today()
    dataset = yf.download(stock_name, start=start, end=end)
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    return close

### State creator

In [9]:
def state_creator(data, timestep, window_size):#Dataset, fecha de prediccion, dias previos a la prediccion
  
  starting_id = timestep - window_size + 1#Primer dato
  
  if starting_id >= 0:
    windowed_data = list(data[starting_id:timestep+1])
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))#Normalizando diferencia entre los valores del dia siguiente y actual
    
  return np.array([state])

### Cargar una divisa de mercado

In [10]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%***********************]  1 of 1 completed


In [11]:
data.head()

Date
2021-01-04    129.410004
2021-01-05    131.009995
2021-01-06    126.599998
2021-01-07    130.919998
2021-01-08    132.050003
Name: Close, dtype: float64

In [12]:
data.tail()

Date
2023-07-21    191.940002
2023-07-24    192.750000
2023-07-25    193.619995
2023-07-26    194.500000
2023-07-27    193.220001
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [13]:
window_size = 10 #Numeros de dias
episodes = 1000  #Epocs

batch_size = 32  #Actualizar red despues de n datos
data_samples = len(data) - 1 #Menos ultimo dia

### Definir el modelo del AI Trader

In [14]:
trader = AI_Trader(window_size)

In [15]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
    
    print("Episodio: {}/{}".format(episode, episodes))
    
    state = state_creator(data, 0, window_size + 1) #Estado inicial
    
    total_profit = 0
    trader.inventory = []#Almacenar acciones. No importa el pasado
    
    for t in tqdm(range(data_samples)):
        
        action = trader.trade(state)#Accion
        
        next_state = state_creator(data, t+1, window_size + 1)#Siguiente estadp,
        reward = 0
        
        if( action == 1): #Compra
            trader.inventory.append(data[t])#Añadir precio a la compra
            print("AI Trader compró: ", stocks_price_format(data[t]))
            
        elif( action == 2 and len(trader.inventory) > 0): #Vende
            buy_price = trader.inventory.pop(0)
            
            reward = max(data[t] - buy_price, 0)#Solo recompensa positias, o en caso de perdida
            total_profit += data[t] - buy_price
            print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price))
        
        if(t ==  (data_samples - 1)):
            done = True
        else:
            done = False
        
        trader.memory.append((state, action, reward, next_state, done))#Añadir a la memoria
        
        state = next_state
        
        if done:
            print("########################")
            print("BENEFICIO TOTAL: {}".format(total_profit))
            print("########################")
                  
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
    
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))
    

Episodio: 1/1000


  0%|                                                                                          | 0/644 [00:00<?, ?it/s]

AI Trader compró:  $ 131.009995
AI Trader vendió:  $ 126.599998  Beneficio: - $ 4.409996
AI Trader compró:  $ 130.919998
AI Trader compró:  $ 132.050003
AI Trader compró:  $ 128.979996
AI Trader compró:  $ 128.800003
AI Trader compró:  $ 130.889999
AI Trader compró:  $ 127.830002
AI Trader compró:  $ 132.029999
AI Trader compró:  $ 136.869995
AI Trader vendió:  $ 139.070007  Beneficio: $ 8.150009
AI Trader compró:  $ 142.919998
AI Trader vendió:  $ 142.059998  Beneficio: $ 10.009995
AI Trader compró:  $ 131.960007
AI Trader vendió:  $ 134.139999  Beneficio: $ 5.160004
AI Trader vendió:  $ 134.990005  Beneficio: $ 6.190002
AI Trader vendió:  $ 133.940002  Beneficio: $ 3.050003
AI Trader compró:  $ 137.389999
AI Trader compró:  $ 136.759995
AI Trader vendió:  $ 136.009995  Beneficio: $ 8.179993
AI Trader compró:  $ 135.389999
AI Trader vendió:  $ 135.369995  Beneficio: $ 3.339996
AI Trader compró:  $ 129.710007


  5%|████▏                                                                            | 33/644 [00:07<02:20,  4.36it/s]

AI Trader vendió:  $ 126.000000  Beneficio: - $ 10.869995


...

  8%|██████▍                                                                        | 52/644 [05:04<1:06:15,  6.72s/it]

AI Trader vendió:  $ 119.989998  Beneficio: - $ 1.040001


 20%|███████████████▋                                                              | 130/644 [14:23<1:00:34,  7.07s/it]

AI Trader compró:  $ 144.500000


 25%|███████████████████▉                                                            | 160/644 [17:59<58:16,  7.22s/it]